In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate
from sklearn import preprocessing

In [2]:
def getDemoPaths(directory):
    demoPaths = []
    for filename in os.listdir(directory):
        demoPaths.append(os.path.join(directory, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    killData, roundData, damageData = [], [], [],
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (killData, roundData, damageData)

In [5]:
def createSidedCSVDFWithScaledPer30(players, damageData, killData, roundData, teams):
    locationNames = []
    playerNames = []

    Kills = []
    killsPer30 = []
    Deaths = []
    deathsPer30 = []
    KdDifferential = []
    kdPer30 = []
    DamageDealt = []
    damageDealtPer30 = []
    DamageTaken = []
    damageTakenPer30 = []
    DamageDifferential = []
    damageDiffPer30 = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            for y in teams.keys():
                if x in teams[y]['Players']:
                    totalRounds.append(teams[y]['CT'])
                    totalRounds.append(teams[y]['T'])
                    break
            
            damageDiffPer30.append(tempCTDamageDifferential / totalRounds[-2] * 30)
            damageDiffPer30.append(tempTDamageDifferential / totalRounds[-1] * 30)

            kdPer30.append(tempCTKdDifferential / totalRounds[-2] * 30)
            kdPer30.append(tempTKdDifferential / totalRounds[-1] * 30)
            
            
            killsPer30.append(tempCTKills / totalRounds[-2] * 30)
            killsPer30.append(tempTKills / totalRounds[-1] * 30)
            
            deathsPer30.append(tempCTDeaths / totalRounds[-2] * 30)
            deathsPer30.append(tempTDeaths / totalRounds[-1] * 30)

            damageDealtPer30.append(tempCTDamageDealt / totalRounds[-2] * 30)
            damageDealtPer30.append(tempTDamageDealt / totalRounds[-1] * 30)
            
            damageTakenPer30.append(tempCTDamageTaken / totalRounds[-2] * 30)
            damageTakenPer30.append(tempTDamageTaken / totalRounds[-1] * 30)
            
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    dataframe['Damage Diff Per 30'] = damageDiffPer30
    dataframe['KD Diff Per 30'] = kdPer30
    dataframe['Kills Per 30'] = killsPer30
    dataframe['Deaths Per 30'] = deathsPer30
    dataframe['Damage Dealt Per 30'] = damageDealtPer30
    dataframe['Damage Taken Per 30'] = damageTakenPer30   
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')

In [6]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [7]:
demo_paths = "/mnt/d/Iem_summer_2021"

In [8]:
demoPaths = getDemoPaths(demo_paths)

In [9]:
demos = []
for x in demoPaths:
    if "ancient" not in x:
        demos.append(x)

In [11]:
demos[27]

'/mnt/d/Iem_summer_2021/og-vs-virtus-pro-m1-mirage_64917.dem'

In [12]:
del demos[27:]

In [13]:
demos

['/mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m1-inferno_64914.dem',
 '/mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m2-overpass_64914.dem',
 '/mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m3-nuke_64914.dem',
 '/mnt/d/Iem_summer_2021/complexity-vs-gambit-m1-mirage_64931.dem',
 '/mnt/d/Iem_summer_2021/complexity-vs-gambit-m2-overpass_64931.dem',
 '/mnt/d/Iem_summer_2021/complexity-vs-gambit-m3-vertigo_64931.dem',
 '/mnt/d/Iem_summer_2021/evil-geniuses-vs-og-m1-dust2_64935.dem',
 '/mnt/d/Iem_summer_2021/evil-geniuses-vs-og-m2-overpass_64935.dem',
 '/mnt/d/Iem_summer_2021/evil-geniuses-vs-og-m3-inferno_64935.dem',
 '/mnt/d/Iem_summer_2021/g2-vs-gambit-m1-vertigo_65082.dem',
 '/mnt/d/Iem_summer_2021/gambit-vs-astralis-m1-inferno_64918.dem',
 '/mnt/d/Iem_summer_2021/gambit-vs-astralis-m2-overpass_64918.dem',
 '/mnt/d/Iem_summer_2021/gambit-vs-evil-geniuses-m1-dust2_65056.dem',
 '/mnt/d/Iem_summer_2021/gambit-vs-evil-geniuses-m2-overpass_65056.dem',
 '/mnt/d/Iem_summer

In [14]:
mirageMappingPath = './mappings/mappings/'
infernoMappingPath = './mappings/mappings/de_inferno_place_mappings.txt'

In [15]:
file = open(infernoMappingPath, 'r')
lines = file.readlines()
file.close()

In [16]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [17]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
        mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [18]:
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[1])

In [19]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [100]:
mappingFile

'./mappings/mappings/de_mirage_place_mappings.txt'

In [20]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [21]:
mappedPlaceName = {}
for x in lines:
    if(x != ''):
        mappedPlaceName[x.split(':')[0]] = x.split(':')[1]

In [22]:
parsers = generateDemoParsers(demos)

15:39:35 [INFO] Go version>=1.14.0
15:39:35 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m1-inferno_64914.dem
15:39:35 [INFO] Setting demo id to complexity-vs-funplus-phoenix-m1-inferno
15:39:35 [INFO] Setting parse rate to 128
15:39:35 [INFO] Go version>=1.14.0
15:39:35 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m2-overpass_64914.dem
15:39:35 [INFO] Setting demo id to complexity-vs-funplus-phoenix-m2-overpass
15:39:35 [INFO] Setting parse rate to 128
15:39:35 [INFO] Go version>=1.14.0
15:39:35 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m3-nuke_64914.dem
15:39:35 [INFO] Setting demo id to complexity-vs-funplus-phoenix-m3-nuke
15:39:35 [INFO] Setting parse rate to 128
15:39:35 [INFO] Go version>=1.14.0
15:39:35 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Iem_summer_2021/complexity-vs-gambit-m1-mirage_64931.

# Demo Parsers: 27


In [23]:
kills, rounds, damages = generateData(parsers)

15:39:38 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
15:39:38 [INFO] Looking for file at /mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m1-inferno_64914.dem
15:40:00 [INFO] Wrote demo parse output to complexity-vs-funplus-phoenix-m1-inferno.json
15:40:00 [INFO] Reading in JSON from complexity-vs-funplus-phoenix-m1-inferno.json
15:40:00 [INFO] JSON data loaded, available in the `json` attribute to parser
15:40:00 [INFO] Successfully parsed JSON output
15:40:00 [INFO] Successfully returned JSON output
15:40:00 [INFO] Parsed kills to Pandas DataFrame
15:40:00 [INFO] Parsed rounds to Pandas DataFrame
15:40:00 [INFO] Parsed damages to Pandas DataFrame
15:40:00 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
15:40:00 [INFO] Looking for file at /mnt/d/Iem_summer_2021/complexity-vs-funplus-phoenix-m2-overpass_64914.dem
15:40:29 [INFO] Wrote demo parse output to complexity-vs-funplus-phoenix-m2-overpass.json
15:40:29 [INFO] Reading i

15:43:59 [INFO] Reading in JSON from gambit-vs-evil-geniuses-m1-dust2.json
15:43:59 [INFO] JSON data loaded, available in the `json` attribute to parser
15:43:59 [INFO] Successfully parsed JSON output
15:43:59 [INFO] Successfully returned JSON output
15:43:59 [INFO] Parsed kills to Pandas DataFrame
15:43:59 [INFO] Parsed rounds to Pandas DataFrame
15:43:59 [INFO] Parsed damages to Pandas DataFrame
15:43:59 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
15:43:59 [INFO] Looking for file at /mnt/d/Iem_summer_2021/gambit-vs-evil-geniuses-m2-overpass_65056.dem
15:44:26 [INFO] Wrote demo parse output to gambit-vs-evil-geniuses-m2-overpass.json
15:44:26 [INFO] Reading in JSON from gambit-vs-evil-geniuses-m2-overpass.json
15:44:27 [INFO] JSON data loaded, available in the `json` attribute to parser
15:44:27 [INFO] Successfully parsed JSON output
15:44:27 [INFO] Successfully returned JSON output
15:44:27 [INFO] Parsed kills to Pandas DataFrame
15:44:27 [INFO] Parsed r

15:48:00 [INFO] Parsed damages to Pandas DataFrame
15:48:00 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
15:48:00 [INFO] Looking for file at /mnt/d/Iem_summer_2021/og-vs-gambit-m2-dust2_65100.dem
15:48:16 [INFO] Wrote demo parse output to og-vs-gambit-m2-dust2.json
15:48:16 [INFO] Reading in JSON from og-vs-gambit-m2-dust2.json
15:48:16 [INFO] JSON data loaded, available in the `json` attribute to parser
15:48:16 [INFO] Successfully parsed JSON output
15:48:16 [INFO] Successfully returned JSON output
15:48:16 [INFO] Parsed kills to Pandas DataFrame
15:48:16 [INFO] Parsed rounds to Pandas DataFrame
15:48:16 [INFO] Parsed damages to Pandas DataFrame
15:48:16 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
15:48:16 [INFO] Looking for file at /mnt/d/Iem_summer_2021/og-vs-gambit-m3-overpass_65100.dem
15:48:38 [INFO] Wrote demo parse output to og-vs-gambit-m3-overpass.json
15:48:38 [INFO] Reading in JSON from og-vs-gambit-m3-overpass.jso

In [186]:
kill = concat_data(kills)
rounds_ = concat_data(rounds)
damage = concat_data(damages)

In [187]:
mirageKill = kill[kill.MapName == 'de_mirage']
infernoKill = kill[kill.MapName == 'de_inferno']
kill = kill[(kill['MapName'] != 'de_mirage') & (kill['MapName'] != 'de_inferno')]

In [188]:
mirageDamage = damage[damage.MapName == 'de_mirage']
infernoDamage = damage[damage.MapName == 'de_inferno']
damage = damage[(damage['MapName'] != 'de_mirage') & (damage['MapName'] != 'de_inferno')]

In [189]:
len(mirageKill)

1088

In [190]:
mirageRound = rounds_[rounds_.MapName == 'de_mirage']
infernoRound = rounds_[rounds_.MapName == 'de_inferno']
rounds_ = rounds_[(rounds_['MapName'] != 'de_mirage') & (rounds_['MapName'] != 'de_inferno')]

In [191]:
mappedPlaceName

{'AbovePit': 'Top Balcony',
 'Admin': 'T Palace Stairs Peek',
 'PalaceInterior': 'Deep Palace',
 'Airplane': 'T Spawn',
 'Alley': 'Deep T Spawn T Ramp',
 'APlatform': 'Stairs Smoke Platform',
 'Atrium': 'T Spawn A Ramp',
 'Attic': 'A Ramp',
 'Back': 'Close A Ramp',
 'BackAlley': 'Close Left CT A Ramp',
 'BackAlleys': 'Tetris',
 'BackCourtyard': 'A Site',
 'BackDoor': 'Jungle Plant',
 'BackEntrance': 'A Site Exposed to Stairs Connector Cat',
 'BackHall': 'Triple Box',
 'BackofA': 'Ninja',
 'BackofB': 'Default A',
 'BackRoad': 'Firebox',
 'BackRoom': 'Under balcony',
 'BackWay': 'Under balcony on top of the little box',
 'BackYard': 'Balcony Boaxes',
 'Banana': 'Sandwhich',
 'BankExterior': 'Ticketbooth ',
 'BankInterior': 'CT Left Side',
 'Barn': 'Deep CT',
 'BarnRoof': 'CT Spawn',
 'Basement': 'CT Murder Hole',
 'Bathroom': 'CT Outside Market',
 'BDoors': 'Bench Closer To Jungle',
 'Beach': 'Bench Closer to Tribox',
 'Bedroom': 'Flowerpot CT',
 'BigForest': 'CT Cheese Spot Murder Hole 

In [192]:
mirageKill['AttackerAreaName'].replace(mappedPlaceName, inplace=True)
mirageKill['VictimAreaName'].replace(mappedPlaceName, inplace=True)

In [193]:
mirageDamage['AttackerAreaName'].replace(mappedPlaceName, inplace=True)
mirageDamage['VictimAreaName'].replace(mappedPlaceName, inplace=True)

In [194]:
mappedPlaceName

{'AbovePit': 'Top Balcony',
 'Admin': 'T Palace Stairs Peek',
 'PalaceInterior': 'Deep Palace',
 'Airplane': 'T Spawn',
 'Alley': 'Deep T Spawn T Ramp',
 'APlatform': 'Stairs Smoke Platform',
 'Atrium': 'T Spawn A Ramp',
 'Attic': 'A Ramp',
 'Back': 'Close A Ramp',
 'BackAlley': 'Close Left CT A Ramp',
 'BackAlleys': 'Tetris',
 'BackCourtyard': 'A Site',
 'BackDoor': 'Jungle Plant',
 'BackEntrance': 'A Site Exposed to Stairs Connector Cat',
 'BackHall': 'Triple Box',
 'BackofA': 'Ninja',
 'BackofB': 'Default A',
 'BackRoad': 'Firebox',
 'BackRoom': 'Under balcony',
 'BackWay': 'Under balcony on top of the little box',
 'BackYard': 'Balcony Boaxes',
 'Banana': 'Sandwhich',
 'BankExterior': 'Ticketbooth ',
 'BankInterior': 'CT Left Side',
 'Barn': 'Deep CT',
 'BarnRoof': 'CT Spawn',
 'Basement': 'CT Murder Hole',
 'Bathroom': 'CT Outside Market',
 'BDoors': 'Bench Closer To Jungle',
 'Beach': 'Bench Closer to Tribox',
 'Bedroom': 'Flowerpot CT',
 'BigForest': 'CT Cheese Spot Murder Hole 

In [195]:
mirageKill

,Tick,Second,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,AttackerX,AttackerY,AttackerZ,AttackerAreaId,...,ThruSmoke,Distance,IsTrade,PlayerTradedName,PlayerTradedTeam,PlayerTradedSteamId,Weapon,RoundNum,MatchId,MapName
578,20081,17.929134,7.656120e+16,poizon,Complexity,CT,66.925705,-2058.303467,-35.968750,47.0,...,False,6.990760,False,None,None,NaN,MP9,2,complexity-vs-gambit-m1-mirage,de_mirage
579,20320,19.811024,7.656120e+16,poizon,Complexity,CT,16.000418,-2289.981445,-39.968750,201.0,...,False,7.479852,False,nafany,Gambit Esports,7.656120e+16,MP9,2,complexity-vs-gambit-m1-mirage,de_mirage
580,20487,21.125984,7.656120e+16,jks,Complexity,CT,-590.391357,-2334.727783,-171.832413,30.0,...,False,28.499464,False,interz,Gambit Esports,7.656120e+16,FAMAS,2,complexity-vs-gambit-m1-mirage,de_mirage
581,20903,24.401575,7.656120e+16,poizon,Complexity,CT,113.228775,-2342.506592,-39.968750,155.0,...,False,10.968264,False,Hobbit,Gambit Esports,7.656120e+16,MP9,2,complexity-vs-gambit-m1-mirage,de_mirage
582,21131,26.196850,7.656120e+16,SH1R0,Gambit Esports,T,-708.754395,-1064.925049,-199.502548,226.0,...,False,8.572980,False,Ax1Le,Gambit Esports,7.656120e+16,Desert Eagle,2,complexity-vs-gambit-m1-mirage,de_mirage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,503265,81.984375,7.656120e+16,Hobbit,Gambit Esports,CT,-233.480896,-2362.971680,-167.968750,413.0,...,False,18.298031,False,nafany,Gambit Esports,7.656120e+16,FAMAS,30,og-vs-gambit-m1-mirage,de_mirage
4283,503923,87.125000,7.656120e+16,flameZ,OG Esports,T,-233.493134,-1623.969482,-167.971161,4.0,...,False,18.609053,False,niko,OG Esports,7.656120e+16,AK-47,30,og-vs-gambit-m1-mirage,de_mirage
4284,504144,88.851562,7.656120e+16,Ax1Le,Gambit Esports,CT,-1412.699463,-970.228088,-167.968750,103.0,...,False,3.322532,False,Hobbit,Gambit Esports,7.656120e+16,AUG,30,og-vs-gambit-m1-mirage,de_mirage
4285,505110,96.398438,7.656120e+16,Ax1Le,Gambit Esports,CT,-917.400269,-1370.068237,-167.968750,289.0,...,False,18.832666,False,valde,OG Esports,7.656120e+16,AK-47,30,og-vs-gambit-m1-mirage,de_mirage


In [196]:
mirageKill.AttackerAreaName.value_counts()

Ticketbooth                               69
Top Balcony                               42
CT Side Apps                              36
Bottom Stairs                             35
Default A                                 35
                                          ..
Inside Market Doors                        1
Close Market Window                        1
Under balcony on top of the little box     1
Headshot Angle Close A Ramp                1
Stairs Smoke Platform                      1
Name: AttackerAreaName, Length: 96, dtype: int64

In [197]:
teams = {}
for x in mirageRound.WinningTeam.unique():
    teams[x] = {'CT' : 0, 'T' : 0}
print(teams.keys())
for i, r in mirageRound.iterrows():
    teams[r['WinningTeam']][r['WinningSide']] += 1
    if(r['WinningSide'] == 'CT'):
        teams[r['LosingTeam']]['T'] += 1
    else:
        teams[r['LosingTeam']]['CT'] += 1


dict_keys(['Complexity', 'Gambit Esports', 'Heroic', 'Evil Geniuses', 'Team Spirit', 'Imperial Sportsbet', 'Ninjas in Pyjamas', 'Virtus.pro', 'OG Esports'])


In [198]:
for x in teams.keys():
    teams[x]['Players'] = []

In [199]:
teams

{'Complexity': {'CT': 15, 'T': 8, 'Players': []},
 'Gambit Esports': {'CT': 23, 'T': 30, 'Players': []},
 'Heroic': {'CT': 32, 'T': 33, 'Players': []},
 'Evil Geniuses': {'CT': 15, 'T': 14, 'Players': []},
 'Team Spirit': {'CT': 15, 'T': 5, 'Players': []},
 'Imperial Sportsbet': {'CT': 5, 'T': 15, 'Players': []},
 'Ninjas in Pyjamas': {'CT': 33, 'T': 33, 'Players': []},
 'Virtus.pro': {'CT': 15, 'T': 15, 'Players': []},
 'OG Esports': {'CT': 15, 'T': 15, 'Players': []}}

In [200]:
for x in mirageKill.AttackerName.unique():
    if x != None and x not in teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players']:
        teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players'].append(x)

In [201]:
player_names = list(mirageKill.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [202]:
player_name = {'Ax1Le' : ['Ax1Le']}

In [203]:
player_name

{'Ax1Le': ['Ax1Le']}

In [204]:
damageDifferentials = createSidedCSVDFWithScaledPer30(player_name, mirageDamage, mirageKill, mirageRound, teams)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Serie

In [205]:
damageDifferentials

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds,Damage Diff Per 30,KD Diff Per 30,Kills Per 30,Deaths Per 30,Damage Dealt Per 30,Damage Taken Per 30
0,Deep Catwalk,Ax1Le,7,3,4,824,395,429,CT,23,559.565217,5.217391,9.130435,3.913043,1074.782609,515.217391
1,Deep Catwalk,Ax1Le,0,0,0,0,0,0,T,30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,CT Side Apps,Ax1Le,0,0,0,0,0,0,CT,23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,CT Side Apps,Ax1Le,4,1,3,373,146,227,T,30,227.000000,3.000000,4.000000,1.000000,373.000000,146.000000
4,Default A,Ax1Le,0,0,0,69,26,43,CT,23,56.086957,0.000000,0.000000,0.000000,90.000000,33.913043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Deep Palace,Ax1Le,0,1,-1,12,100,-88,T,30,-88.000000,-1.000000,0.000000,1.000000,12.000000,100.000000
84,Snipers Nest,Ax1Le,0,0,0,0,0,0,CT,23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
85,Snipers Nest,Ax1Le,0,0,0,0,16,-16,T,30,-16.000000,0.000000,0.000000,0.000000,0.000000,16.000000
86,Deep CT,Ax1Le,0,0,0,0,0,0,CT,23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [206]:
damageDifferentials.to_csv('./damageDifferentials/ax1le_iem_summer/mirage.csv')

In [ ]:
player_name['Ax1le']